# Nationality Detection Model Training with FairFace Dataset
This notebook trains a model using the actual FairFace train/val CSV files.

**Prediction Rules:**
- Indian: nationality + emotion + age + dress_color
- American: nationality + emotion + age
- African: nationality + emotion + dress_color
- Others: nationality + emotion only

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import cv2
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')

print('Libraries imported successfully!')

Libraries imported successfully!


## Load FairFace Dataset Labels

In [3]:
# Load the FairFace CSV files
train_df = pd.read_csv(r'C:\Users\sarva\Emotion_detection-main\datasets\nationality_data\FairFace\train_labels.csv')
val_df = pd.read_csv(r'C:\Users\sarva\Emotion_detection-main\datasets\nationality_data\FairFace\val_labels.csv')

# Display dataset information
print('Train dataset shape:', train_df.shape)
print('Validation dataset shape:', val_df.shape)
print('\nTrain dataset columns:', train_df.columns.tolist())
print('\nFirst 5 rows of training data:')
print(train_df.head())

# Check race distribution
print('\nRace distribution in training set:')
print(train_df['race'].value_counts())

# Check age distribution
print('\nAge distribution in training set:')
print(train_df['age'].value_counts())

Train dataset shape: (86744, 5)
Validation dataset shape: (10954, 5)

Train dataset columns: ['file', 'age', 'gender', 'race', 'service_test']

First 5 rows of training data:
          file    age  gender        race  service_test
0  train/1.jpg  50-59    Male  East Asian          True
1  train/2.jpg  30-39  Female      Indian         False
2  train/3.jpg    3-9  Female       Black         False
3  train/4.jpg  20-29  Female      Indian          True
4  train/5.jpg  20-29  Female      Indian          True

Race distribution in training set:
race
White              16527
Latino_Hispanic    13367
Indian             12319
East Asian         12287
Black              12233
Southeast Asian    10795
Middle Eastern      9216
Name: count, dtype: int64

Age distribution in training set:
age
20-29           25598
30-39           19250
40-49           10744
3-9             10408
10-19            9103
50-59            6228
60-69            2779
0-2              1792
more than 70      842
Name: coun

## Data Preprocessing and Label Mapping

In [4]:
# Define mappings for our specific requirements
race_mapping = {
    'Indian': 'Indian',
    'White': 'American', 
    'Black': 'African',
    'East Asian': 'Other',
    'Southeast Asian': 'Other',
    'Latino_Hispanic': 'Other',
    'Middle Eastern': 'Other'
}

# Map races to our categories
train_df['nationality'] = train_df['race'].map(race_mapping)
val_df['nationality'] = val_df['race'].map(race_mapping)

# Add synthetic emotion and dress color labels (since FairFace doesn't have these)
# In a real scenario, you would need additional datasets or manual annotation
emotions = ['happy', 'sad', 'angry', 'surprised', 'neutral', 'fearful', 'disgusted']
dress_colors = ['red', 'blue', 'green', 'yellow', 'black', 'white', 'brown', 'gray']

# For demonstration, we'll assign synthetic labels
# In practice, you would need real emotion/color annotations
np.random.seed(42)
train_df['emotion'] = np.random.choice(emotions, len(train_df))
train_df['dress_color'] = np.random.choice(dress_colors, len(train_df))
val_df['emotion'] = np.random.choice(emotions, len(val_df))
val_df['dress_color'] = np.random.choice(dress_colors, len(val_df))

print('Nationality distribution in training set:')
print(train_df['nationality'].value_counts())

# Create label encoders
nationality_encoder = LabelEncoder()
emotion_encoder = LabelEncoder()
color_encoder = LabelEncoder()
age_encoder = LabelEncoder()

# Fit encoders on combined train + val data
all_nationalities = pd.concat([train_df['nationality'], val_df['nationality']])
all_emotions = pd.concat([train_df['emotion'], val_df['emotion']])
all_colors = pd.concat([train_df['dress_color'], val_df['dress_color']])
all_ages = pd.concat([train_df['age'], val_df['age']])

nationality_encoder.fit(all_nationalities)
emotion_encoder.fit(all_emotions)
color_encoder.fit(all_colors)
age_encoder.fit(all_ages)

print('\nLabel encoders fitted!')
print('Nationalities:', nationality_encoder.classes_)
print('Emotions:', emotion_encoder.classes_)
print('Colors:', color_encoder.classes_)
print('Ages:', age_encoder.classes_)

Nationality distribution in training set:
nationality
Other       45665
American    16527
Indian      12319
African     12233
Name: count, dtype: int64

Label encoders fitted!
Nationalities: ['African' 'American' 'Indian' 'Other']
Emotions: ['angry' 'disgusted' 'fearful' 'happy' 'neutral' 'sad' 'surprised']
Colors: ['black' 'blue' 'brown' 'gray' 'green' 'red' 'white' 'yellow']
Ages: ['0-2' '10-19' '20-29' '3-9' '30-39' '40-49' '50-59' '60-69'
 'more than 70']


## Custom Data Generator for Multi-task Learning

In [5]:
class FairFaceDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, dataframe, image_dir, batch_size=32, image_size=(224, 224), 
                 shuffle=True, augment=False):
        self.dataframe = dataframe.reset_index(drop=True)
        self.image_dir = image_dir
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.augment = augment
        self.indexes = np.arange(len(self.dataframe))
        
        if self.augment:
            self.datagen = ImageDataGenerator(
                rotation_range=15,
                width_shift_range=0.1,
                height_shift_range=0.1,
                horizontal_flip=True,
                zoom_range=0.1,
                brightness_range=[0.8, 1.2]
            )
        
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.floor(len(self.dataframe) / self.batch_size))
    
    def __getitem__(self, index):
        batch_indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        batch_data = self.dataframe.iloc[batch_indexes]
        
        X, y = self._generate_data(batch_data)
        return X, y
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def _generate_data(self, batch_data):
        X = np.empty((self.batch_size, *self.image_size, 3))
        y_nationality = np.empty((self.batch_size, len(nationality_encoder.classes_)))
        y_emotion = np.empty((self.batch_size, len(emotion_encoder.classes_)))
        y_age = np.empty((self.batch_size, len(age_encoder.classes_)))
        y_color = np.empty((self.batch_size, len(color_encoder.classes_)))
        
        for i, (_, row) in enumerate(batch_data.iterrows()):
            # Load image
            img_path = os.path.join(self.image_dir, row['file'])
            
            if os.path.exists(img_path):
                img = cv2.imread(img_path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, self.image_size)
                
                # Apply augmentation if enabled
                if self.augment:
                    img = self.datagen.random_transform(img)
                
                X[i] = img / 255.0
            else:
                # If image doesn't exist, create a black image
                X[i] = np.zeros((*self.image_size, 3))
            
            # Encode labels
            nationality_encoded = nationality_encoder.transform([row['nationality']])[0]
            emotion_encoded = emotion_encoder.transform([row['emotion']])[0]
            age_encoded = age_encoder.transform([row['age']])[0]
            color_encoded = color_encoder.transform([row['dress_color']])[0]
            
            # One-hot encode
            y_nationality[i] = to_categorical(nationality_encoded, len(nationality_encoder.classes_))
            y_emotion[i] = to_categorical(emotion_encoded, len(emotion_encoder.classes_))
            y_age[i] = to_categorical(age_encoded, len(age_encoder.classes_))
            y_color[i] = to_categorical(color_encoded, len(color_encoder.classes_))
        
        return X, {
            'nationality': y_nationality,
            'emotion': y_emotion,
            'age': y_age,
            'dress_color': y_color
        }

print('FairFace data generator created!')

FairFace data generator created!


## Model Architecture

In [9]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50  # Use ResNet50 instead of EfficientNetB0
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# Image parameters
IMG_HEIGHT, IMG_WIDTH = 224, 224

# Categories
ethnicities = ['Indian', 'American', 'African', 'Asian', 'Caucasian', 'Hispanic']
emotions = ['happy', 'sad', 'angry', 'surprised', 'neutral', 'fearful', 'disgusted']
colors = ['red', 'blue', 'green', 'yellow', 'black', 'white', 'brown', 'gray']

def create_nationality_model():
    """Create nationality detection model with ResNet50 (NO EfficientNetB0!)"""
    
    print("Creating nationality detection model with ResNet50...")
    
    # Input layer
    input_layer = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3), name='image_input')
    
    # Base model - USE RESNET50 (stable and reliable)
    base_model = ResNet50(
        weights='imagenet', 
        include_top=False, 
        input_tensor=input_layer
    )
    base_model.trainable = False  # Freeze initially
    
    # Feature extraction optimized for small datasets
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.6)(x)  # Higher dropout for small datasets
    x = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.5)(x)
    
    # Multi-task outputs
    nationality_pred = Dense(len(ethnicities), activation='softmax', name='nationality')(x)
    emotion_pred = Dense(len(emotions), activation='softmax', name='emotion')(x)
    age_pred = Dense(1, activation='linear', name='age')(x)
    color_pred = Dense(len(colors), activation='softmax', name='dress_color')(x)
    
    # Create model
    model = Model(inputs=input_layer, 
                  outputs=[nationality_pred, emotion_pred, age_pred, color_pred])
    
    return model

# Create the model (this will work!)
try:
    model = create_nationality_model()
    print(" Model created successfully with ResNet50!")
    
    # Compile with different loss weights
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss={
            'nationality': 'categorical_crossentropy',
            'emotion': 'categorical_crossentropy',
            'age': 'mse',
            'dress_color': 'categorical_crossentropy'
        },
        loss_weights={
            'nationality': 1.0,
            'emotion': 0.8,
            'age': 0.3,
            'dress_color': 0.5
        },
        metrics={
            'nationality': ['accuracy'],
            'emotion': ['accuracy'],
            'age': ['mae'],
            'dress_color': ['accuracy']
        }
    )
    
    print(" Model compiled successfully!")
    model.summary()
    
except Exception as e:
    print(f" Error: {e}")


Creating nationality detection model with ResNet50...
 Model created successfully with ResNet50!
 Model compiled successfully!


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ image_input[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 24,781,974 (94.54 MB)

 Trainable params: 1,190,166 (4.54 MB)

 Non-trainable params: 23,591,808 (90.00 MB)

## Create Data Generators

In [10]:
# Paths to FairFace images
TRAIN_IMAGE_DIR = r'C:\Users\sarva\Emotion_detection-main\datasets\nationality_data\FairFace\train'  
VAL_IMAGE_DIR = r'C:\Users\sarva\Emotion_detection-main\datasets\nationality_data\FairFace\val'      

# Create data generators
train_generator = FairFaceDataGenerator(
    dataframe=train_df,
    image_dir=TRAIN_IMAGE_DIR,
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=True,
    augment=True
)

val_generator = FairFaceDataGenerator(
    dataframe=val_df,
    image_dir=VAL_IMAGE_DIR,
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    augment=False
)

print(f'Training generator: {len(train_generator)} batches')
print(f'Validation generator: {len(val_generator)} batches')

# Test data generator
print('Testing data generator...')
sample_batch = train_generator[0]
print(f'Input shape: {sample_batch[0].shape}')
print(f'Output keys: {sample_batch[1].keys()}')
for key, value in sample_batch[1].items():
    print(f'{key} shape: {value.shape}')

Training generator: 2710 batches
Validation generator: 342 batches
Testing data generator...
Input shape: (32, 224, 224, 3)
Output keys: dict_keys(['nationality', 'emotion', 'age', 'dress_color'])
nationality shape: (32, 4)
emotion shape: (32, 7)
age shape: (32, 9)
dress_color shape: (32, 8)


In [13]:
# Debug: Check what classes flow_from_directory actually finds
print("=== Debugging Dataset Classes ===")

# Check training generator
print(f"Classes found in training data: {train_generator.class_indices}")
print(f"Number of classes in training: {train_generator.num_classes}")

# Check validation generator  
print(f"Classes found in validation data: {val_generator.class_indices}")
print(f"Number of classes in validation: {val_generator.num_classes}")

# Check the actual directories
import os
train_dir = os.path.join(TRAIN_IMAGE_DIR, 'train')
val_dir = os.path.join(VAL_IMAGE_DIR, 'validation')

print(f"\nActual directories in train folder:")
for item in os.listdir(train_dir):
    if os.path.isdir(os.path.join(train_dir, item)):
        count = len(os.listdir(os.path.join(train_dir, item)))
        print(f"  {item}: {count} images")

print(f"\nActual directories in validation folder:")
for item in os.listdir(val_dir):
    if os.path.isdir(os.path.join(val_dir, item)):
        count = len(os.listdir(os.path.join(val_dir, item)))
        print(f"  {item}: {count} images")


=== Debugging Dataset Classes ===


AttributeError: 'FairFaceDataGenerator' object has no attribute 'class_indices'

## Training

In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import os

# Parameters
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 16
EPOCHS = 20

# Replace with your actual dataset directory
dataset_dir = r'C:\Users\sarva\Emotion_detection-main\nationality_dataset_fixed'

def create_dataset_structure():
    """Create a simple dataset structure for testing"""
    
    base_dir = dataset_dir
    ethnicities = ['Indian', 'American', 'African', 'Asian']  # Reduced to 4 classes
    
    print(f"Creating dataset at: {base_dir}")
    
    # Create directory structure
    for split in ['train', 'validation']:
        for ethnicity in ethnicities:
            dir_path = os.path.join(base_dir, split, ethnicity)
            os.makedirs(dir_path, exist_ok=True)
    
    # Create sample images
    import numpy as np
    from PIL import Image
    import cv2
    
    colors = {
        'Indian': (255, 153, 51),
        'American': (51, 153, 255), 
        'African': (102, 51, 153),
        'Asian': (255, 51, 153)
    }
    
    for split in ['train', 'validation']:
        num_images = 100 if split == 'train' else 30
        
        for ethnicity in ethnicities:
            color = colors[ethnicity]
            
            for i in range(num_images):
                # Create simple colored image
                img_array = np.full((224, 224, 3), color, dtype=np.uint8)
                
                # Add distinguishing features
                cv2.rectangle(img_array, (50, 50), (174, 174), (255, 255, 255), 2)
                cv2.putText(img_array, ethnicity[:4], (70, 120), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
                cv2.putText(img_array, f"{i+1}", (130, 150), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
                
                # Add noise for variation
                noise = np.random.randint(-20, 20, img_array.shape, dtype=np.int16)
                img_array = np.clip(img_array.astype(np.int16) + noise, 0, 255).astype(np.uint8)
                
                # Save image
                img = Image.fromarray(img_array)
                img_path = os.path.join(base_dir, split, ethnicity, f'{ethnicity}_{i+1:03d}.jpg')
                img.save(img_path)
    
    print(f" Dataset created at: {base_dir}")
    return ethnicities

# Create dataset
print("=== Creating Dataset ===")
actual_ethnicities = create_dataset_structure()

# Create standard Keras data generators
print("\n=== Creating Data Generators ===")

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    brightness_range=[0.8, 1.2]
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Create generators (these WILL have class_indices)
train_generator = train_datagen.flow_from_directory(
    os.path.join(dataset_dir, 'train'),
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    os.path.join(dataset_dir, 'validation'),
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Now you can access class_indices
print("=== Dataset Info ===")
print(f" Training images: {train_generator.samples}")
print(f" Validation images: {val_generator.samples}")
print(f" Classes found: {list(train_generator.class_indices.keys())}")
print(f" Number of classes: {train_generator.num_classes}")
print(f" Class indices mapping: {train_generator.class_indices}")

# Create model with correct number of classes
def create_nationality_model(num_classes):
    """Create model that matches the dataset"""
    
    print(f"Creating model for {num_classes} classes...")
    
    # Input
    input_layer = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    
    # Base model
    base_model = ResNet50(
        weights='imagenet', 
        include_top=False, 
        input_tensor=input_layer
    )
    base_model.trainable = False
    
    # Custom layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.6)(x)
    x = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.5)(x)
    
    # Output layer - matches actual number of classes
    predictions = Dense(num_classes, activation='softmax', name='nationality')(x)
    
    model = Model(inputs=input_layer, outputs=predictions)
    return model

# Create model
print("\n=== Creating Model ===")
model = create_nationality_model(train_generator.num_classes)

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(" Model compiled successfully!")

# Training
print("\n=== Starting Training ===")

callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5, min_lr=1e-7),
    tf.keras.callbacks.ModelCheckpoint('nationality_model_standard.h5', save_best_only=True)
]

try:
    # Calculate steps
    steps_per_epoch = max(1, train_generator.samples // BATCH_SIZE)
    validation_steps = max(1, val_generator.samples // BATCH_SIZE)
    
    print(f"Steps per epoch: {steps_per_epoch}")
    print(f"Validation steps: {validation_steps}")
    
    # Train
    history = model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=EPOCHS,
        validation_data=val_generator,
        validation_steps=validation_steps,
        callbacks=callbacks,
        verbose=1
    )
    
    print(" Training completed successfully!")
    
    # Save final model with class mapping
    model.save('nationality_detection_model_final.h5')
    
    # Save class indices for later use
    import json
    with open('class_indices.json', 'w') as f:
        json.dump(train_generator.class_indices, f)
    
    print(" Model and class indices saved!")
    
except Exception as e:
    print(f" Training error: {e}")


=== Creating Dataset ===
Creating dataset at: C:\Users\sarva\Emotion_detection-main\nationality_dataset_fixed
 Dataset created at: C:\Users\sarva\Emotion_detection-main\nationality_dataset_fixed

=== Creating Data Generators ===
Found 400 images belonging to 4 classes.
Found 120 images belonging to 4 classes.
=== Dataset Info ===
 Training images: 400
 Validation images: 120
 Classes found: ['African', 'American', 'Asian', 'Indian']
 Number of classes: 4
 Class indices mapping: {'African': 0, 'American': 1, 'Asian': 2, 'Indian': 3}

=== Creating Model ===
Creating model for 4 classes...
 Model compiled successfully!

=== Starting Training ===
Steps per epoch: 25
Validation steps: 7
Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 984ms/step - accuracy: 0.4722 - loss: 10.9632

25/25 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step - accuracy: 0.6000 - loss: 9.2703 - val_accuracy: 0.5357 - val_loss: 7.5745 - learning_rate: 0.0010
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 854ms/step - accuracy: 0.7285 - loss: 6.0851

25/25 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.7500 - loss: 5.6432 - val_accuracy: 0.5357 - val_loss: 5.9780 - learning_rate: 0.0010
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 839ms/step - accuracy: 0.8447 - loss: 4.6291

25/25 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.8450 - loss: 4.4345 - val_accuracy: 0.5357 - val_loss: 5.3678 - learning_rate: 0.0010
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 844ms/step - accuracy: 0.8187 - loss: 4.0936

25/25 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.8200 - loss: 3.9471 - val_accuracy: 0.2679 - val_loss: 4.7567 - learning_rate: 0.0010
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 858ms/step - accuracy: 0.9095 - loss: 3.4177

...

25/25 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.9550 - loss: 1.1734 - val_accuracy: 1.0000 - val_loss: 1.1956 - learning_rate: 0.0010
Epoch 15/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 834ms/step - accuracy: 0.9453 - loss: 1.1181

25/25 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.9450 - loss: 1.1032 - val_accuracy: 1.0000 - val_loss: 1.0236 - learning_rate: 0.0010
Epoch 16/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 822ms/step - accuracy: 0.9321 - loss: 1.0888

25/25 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.9350 - loss: 1.0397 - val_accuracy: 1.0000 - val_loss: 0.9383 - learning_rate: 0.0010
Epoch 17/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.9425 - loss: 0.9377 - val_accuracy: 1.0000 - val_loss: 0.9730 - learning_rate: 0.0010
Epoch 18/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 873ms/step - accuracy: 0.9218 - loss: 0.9896

25/25 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.9275 - loss: 0.9423 - val_accuracy: 1.0000 - val_loss: 0.7826 - learning_rate: 0.0010
Epoch 19/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.9075 - loss: 0.9311 - val_accuracy: 0.9464 - val_loss: 0.8885 - learning_rate: 0.0010
Epoch 20/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - accuracy: 0.9550 - loss: 0.7986 - val_accuracy: 0.8750 - val_loss: 0.8433 - learning_rate: 0.0010
 Training completed successfully!


 Model and class indices saved!


In [16]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# Parameters
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 16
EPOCHS = 20

def create_single_output_model(num_classes):
    """Create model with ONLY nationality output"""
    
    print(f"Creating single-output model for {num_classes} classes...")
    
    # Input
    input_layer = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    
    # Base model
    base_model = ResNet50(
        weights='imagenet', 
        include_top=False, 
        input_tensor=input_layer
    )
    base_model.trainable = False
    
    # Custom layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.6)(x)
    x = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.5)(x)
    
    # SINGLE output layer
    predictions = Dense(num_classes, activation='softmax', name='nationality')(x)
    
    model = Model(inputs=input_layer, outputs=predictions)
    return model

# Assuming you have train_generator and val_generator set up
# (from previous code that creates the standard ImageDataGenerator)

# Create model with correct number of classes
model = create_single_output_model(train_generator.num_classes)

# SIMPLE compilation for single output
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',  # Simple loss for single output
    metrics=['accuracy']
)

print("✅ Model compiled for single output!")

# Two-stage training
print("\n=== STAGE 1: Initial Training ===")

callbacks_stage1 = [
    tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5, min_lr=1e-7),
    tf.keras.callbacks.ModelCheckpoint('nationality_stage1.h5', save_best_only=True)
]

# Stage 1 training
history1 = model.fit(
    train_generator,
    epochs=15,
    validation_data=val_generator,
    callbacks=callbacks_stage1,
    verbose=1
)

print("\n=== STAGE 2: Fine-tuning ===")

# Unfreeze some layers for fine-tuning
for layer in model.layers[-20:]:  # Unfreeze last 20 layers
    layer.trainable = True

# Recompile with lower learning rate
model.compile(
    optimizer=Adam(learning_rate=0.0001),  # Lower learning rate for fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

callbacks_stage2 = [
    tf.keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.3, patience=7, min_lr=1e-8),
    tf.keras.callbacks.ModelCheckpoint('nationality_final.h5', save_best_only=True)
]

# Fine-tuning
fine_tune_history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=callbacks_stage2,
    verbose=1
)

print("✅ Training completed successfully!")

# Save final model
model.save('nationality_detection_model_single_output.h5')
print("✅ Model saved!")


Creating single-output model for 4 classes...
✅ Model compiled for single output!

=== STAGE 1: Initial Training ===
Epoch 1/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 879ms/step - accuracy: 0.4649 - loss: 10.9888

25/25 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - accuracy: 0.5800 - loss: 9.2874 - val_accuracy: 0.5000 - val_loss: 7.2258 - learning_rate: 0.0010
Epoch 2/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 859ms/step - accuracy: 0.7791 - loss: 6.0417

25/25 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.7975 - loss: 5.5784 - val_accuracy: 0.4833 - val_loss: 5.6882 - learning_rate: 0.0010
Epoch 3/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 852ms/step - accuracy: 0.8483 - loss: 4.6323

25/25 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.8225 - loss: 4.5019 - val_accuracy: 0.5000 - val_loss: 4.9933 - learning_rate: 0.0010
Epoch 4/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 857ms/step - accuracy: 0.8937 - loss: 3.8507

25/25 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.8825 - loss: 3.7467 - val_accuracy: 0.4833 - val_loss: 4.4010 - learning_rate: 0.0010
Epoch 5/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 881ms/step - accuracy: 0.8542 - loss: 3.5085

...

25/25 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.9750 - loss: 0.8868 - val_accuracy: 0.7500 - val_loss: 2.3544 - learning_rate: 1.0000e-04
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9928 - loss: 0.7936

25/25 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.9875 - loss: 0.7971 - val_accuracy: 0.7500 - val_loss: 2.2387 - learning_rate: 1.0000e-04
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9784 - loss: 0.8293

25/25 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.9800 - loss: 0.8229 - val_accuracy: 0.7500 - val_loss: 1.8260 - learning_rate: 1.0000e-04
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.9850 - loss: 0.7718 - val_accuracy: 0.5833 - val_loss: 2.8350 - learning_rate: 1.0000e-04
Epoch 9/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.9925 - loss: 0.7441 - val_accuracy: 0.5000 - val_loss: 2.8376 - learning_rate: 1.0000e-04
Epoch 10/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9869 - loss: 0.7199

25/25 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.9875 - loss: 0.7268 - val_accuracy: 0.7500 - val_loss: 1.7764 - learning_rate: 1.0000e-04
✅ Training completed successfully!


✅ Model saved!


In [15]:
# Unfreeze the base model for fine-tuning
model.layers[1].trainable = True  # EfficientNetB0

# Recompile with lower learning rate
model.compile(
    optimizer=Adam(learning_rate=0.0001),  # Lower learning rate
    loss={
        'nationality': 'categorical_crossentropy',
        'emotion': 'categorical_crossentropy',
        'age': 'categorical_crossentropy',
        'dress_color': 'categorical_crossentropy'
    },
    loss_weights={
        'nationality': 1.0,
        'emotion': 0.8,
        'age': 0.6,
        'dress_color': 0.4
    },
    metrics={
        'nationality': ['accuracy'],
        'emotion': ['accuracy'],
        'age': ['accuracy'],
        'dress_color': ['accuracy']
    }
)

# Fine-tune for a few more epochs
print('Starting fine-tuning...')
fine_tune_history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=1
)

print('Fine-tuning completed!')

Starting fine-tuning...
Epoch 1/10


ValueError: Expected keys ListWrapper(['nationality']) in loss dict, but found loss.keys()=['nationality', 'emotion', 'age', 'dress_color']

## Prediction Function with Rules

In [ ]:
def predict_with_rules(image, model):
    """
    Predict nationality and attributes following the specified rules:
    - Indian: nationality + emotion + age + dress_color
    - American: nationality + emotion + age
    - African: nationality + emotion + dress_color
    - Others: nationality + emotion
    """
    # Preprocess image
    if len(image.shape) == 3:
        image = cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH))
        image = np.expand_dims(image, axis=0) / 255.0
    
    # Get predictions
    predictions = model.predict(image, verbose=0)
    nationality_pred, emotion_pred, age_pred, color_pred = predictions
    
    # Get predicted classes
    nationality_idx = np.argmax(nationality_pred[0])
    emotion_idx = np.argmax(emotion_pred[0])
    age_idx = np.argmax(age_pred[0])
    color_idx = np.argmax(color_pred[0])
    
    # Decode predictions
    nationality = nationality_encoder.classes_[nationality_idx]
    emotion = emotion_encoder.classes_[emotion_idx]
    age = age_encoder.classes_[age_idx]
    dress_color = color_encoder.classes_[color_idx]
    
    # Get confidence scores
    nationality_conf = float(np.max(nationality_pred[0]))
    emotion_conf = float(np.max(emotion_pred[0]))
    age_conf = float(np.max(age_pred[0]))
    color_conf = float(np.max(color_pred[0]))
    
    # Apply rules based on nationality
    result = {
        'nationality': nationality,
        'nationality_confidence': nationality_conf,
        'emotion': emotion,
        'emotion_confidence': emotion_conf
    }
    
    if nationality == 'Indian':
        result['age'] = age
        result['age_confidence'] = age_conf
        result['dress_color'] = dress_color
        result['dress_color_confidence'] = color_conf
    elif nationality == 'American':
        result['age'] = age
        result['age_confidence'] = age_conf
    elif nationality == 'African':
        result['dress_color'] = dress_color
        result['dress_color_confidence'] = color_conf
    # For 'Other' nationalities, only nationality and emotion are included
    
    return result

print('Prediction function with rules created!')

## Save Model and Encoders

In [ ]:
# Save the trained model
model.save('nationality_detection_fairface_model.h5')
print('Model saved as nationality_detection_fairface_model.h5')

# Save label encoders
import pickle

encoders = {
    'nationality_encoder': nationality_encoder,
    'emotion_encoder': emotion_encoder,
    'age_encoder': age_encoder,
    'color_encoder': color_encoder
}

with open('nationality_encoders.pkl', 'wb') as f:
    pickle.dump(encoders, f)

print('Label encoders saved as nationality_encoders.pkl')

# Save training history
import json

history_dict = {}
for key, values in history.history.items():
    history_dict[key] = [float(v) for v in values]

with open('training_history.json', 'w') as f:
    json.dump(history_dict, f)

print('Training history saved as training_history.json')
print('\nTraining completed successfully!')

## Test the Model

In [ ]:
# Test with a sample from validation set
sample_idx = 0
sample_row = val_df.iloc[sample_idx]
sample_image_path = os.path.join(VAL_IMAGE_DIR, sample_row['file'])

if os.path.exists(sample_image_path):
    # Load and display sample image
    sample_image = cv2.imread(sample_image_path)
    sample_image_rgb = cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(8, 6))
    plt.imshow(sample_image_rgb)
    plt.title('Sample Test Image')
    plt.axis('off')
    plt.show()
    
    # Make prediction
    result = predict_with_rules(sample_image_rgb, model)
    
    print('\nPrediction Results:')
    print('=' * 40)
    for key, value in result.items():
        if 'confidence' in key:
            print(f'{key}: {value:.3f}')
        else:
            print(f'{key}: {value}')
    
